### Описание задания.

#### Часть 1: Составление словарей для классификации по тональности.

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.

Данные для задания: датасет отзывов на банки с сайта banki.ru. Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5. Тексты хранятся в json-ах в массиве responses.

- Разбить всю коллекцию отзывов на предложения. Лемматизировать все слова.
- Обучить по коллекции предложений word2vec.
- Привести несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    - тест на определение ближайших слов
    - тест на аналогии (мужчина – король : женщина – королева)
    - тест на определение лишнего слова.

- Построить несколько визуализаций:
    - TSNE для топ-100 (или топ-500) слов и найти осмысленные кластеры слов
    - задать координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.


#### Часть 2:  Распространение метки.

Определить 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8 негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустить любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
- graphlab.label_propagation (graphlab доступен бесплатно по образовательной лицензии)
- sklearn.semi_supervised.LabelPropagation
- sklearn.semi_supervised.LabelSpreading

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
import bz2
import igraph as ig
import gdown
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pymorphy2
import re
import regex
import seaborn as sns

from collections import Counter
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from PIL import Image
from scipy import sparse
from string import punctuation
from tqdm import tqdm

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import lsimodel, TfidfModel, word2vec

from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics  import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

#### Необходимые функции

In [5]:
mystopwords = stopwords.words('russian')

ru_words = re.compile("[А-Яа-я]+")

def get_top_word(bow, count_word = 10):
    '''
    Принимает на вход результат TfidfVectorizer.fit_transform.
    Возвращает список ключевых слов
    '''
    idx = np.ravel(bow.sum(axis=0).argsort(axis=1))[::-1][:count_word]
    top_words = np.array(vec.get_feature_names_out())[idx].tolist()
    return top_words


def words_only(text):
    return " ".join(ru_words.findall(text))

def lemmatize(text):
    try:
        return  " ".join([morph.parse(word)[0].normal_form for word in text.lower().split()])
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
 
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

#### Скачивание данных

In [6]:
url = 'https://drive.google.com/uc?id=1OelGGXPXBinXvZnDb1Bmmxe4rtdBmqdk'

In [7]:
gdown.download(url, quiet=True)

'banki_responses.json.bz2'

In [8]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:06, 3033.48it/s]


In [9]:
len(responses)

153499

In [10]:
df = pd.DataFrame(responses)
df.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


### Часть 1: Составление словарей для классификации по тональности.
#### Разбить всю коллекцию отзывов на предложения. Лемматизировать все слова.

In [11]:
sentences = []
for i in range(len(df)):
    sentences.extend(sent_tokenize(df.text.iloc[i]))

In [12]:
sentences[:5]

['Добрый день!',
 'Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.',
 'Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года.',
 'Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован.',
 'Каким образом Вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор, мне по меньшей мере не понятно,\xa0 и почему по Вашей милости я должна переживать и бояться за себе и свое имущество.']

In [13]:
morph = pymorphy2.MorphAnalyzer()

In [14]:
lemmas = []
for sentence in tqdm(sentences):
    lemmas.append(preprocess(sentence))

100%|█████████████████████████████████████████████████████████████████████| 2664065/2664065 [3:39:35<00:00, 202.20it/s]


In [16]:
lemmas[:5]

['добрый день',
 'являться клиент банк поручитель кредитный договор также являться либо лицо письменно оформить отношение банк повод урегулирование чей либо задолженность',
 'начинать март год начать приходить бесконечный письмо ооо прить коллект адрес город москва уваровский переулок дом квартира угроза возбуждение уголовный дело отношение гражданин филиппов эдуард владимирович который уклоняться уплата взять кредит договор год договор год',
 'весь ответственность хотеть заверить вышеуказанный гражданин который разыскивать знак адрес зарегистрировать',
 'образ удостовериться подлинность документ оформить имя кредитный договор малый мера понятно почему ваш милость должный переживать бояться свой имущество']

#### Обучение по коллекции предложений word2vec.

In [24]:
words = [sentence.split() for sentence in lemmas]

In [28]:
model_response = word2vec.Word2Vec(words, workers=4, vector_size=200, min_count=3, window=5, epochs=15)

#### Привести несколько удачных и неудачных примеров решения стандартных текстов для word2vec.

In [34]:
model_response.wv.most_similar('доход')

[('заработок', 0.7398349046707153),
 ('ндфл', 0.6394184231758118),
 ('оклад', 0.5916330814361572),
 ('прибыль', 0.570221483707428),
 ('зарплата', 0.5633077621459961),
 ('зп', 0.5517975687980652),
 ('доходность', 0.5093913674354553),
 ('иждивенец', 0.5003501772880554),
 ('оборот', 0.4870395362377167),
 ('инфляция', 0.48121801018714905)]

In [52]:
model_response.wv.most_similar(positive=['сотрудник', 'человек'], negative=['работник'])

[('парень', 0.5981752276420593),
 ('женщина', 0.5726892352104187),
 ('особа', 0.5610772371292114),
 ('жь', 0.5284541249275208),
 ('мужчина', 0.5239837169647217),
 ('паренёк', 0.515592634677887),
 ('дама', 0.5074449181556702),
 ('мужик', 0.4959198832511902),
 ('девушка', 0.4720597565174103),
 ('барышня', 0.45441964268684387)]

In [40]:
model_response.wv.doesnt_match('вклад прибыль доход рост кредит'.split())

'кредит'

In [50]:
print(model_response.wv.similarity('процент', 'инфляция'))
print(model_response.wv.similarity('заработок', 'инфляция'))

0.3928402
0.46484727


**Вывод:** тесты показали, что модель хорошо обучилась, хотя мы видим, странное слово 'жь' во втором тесте.

### Построение визуализаций:
- TSNE для топ-500 слов и найти осмысленные кластеры слов.

In [65]:
top_words = []
fd = FreqDist()
for word in words:
    fd.update(word)
for idx in fd.most_common(500):
    top_words.append(idx[0])
print(top_words)

['банк', 'карта', 'это', 'деньга', 'день', 'всё', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'мочь', 'счёт', 'свой', 'сказать', 'сумма', 'заявление', 'год', 'вопрос', 'время', 'ещё', 'получить', 'кредитный', 'весь', 'ответ', 'вклад', 'офис', 'телефон', 'платёж', 'договор', 'ваш', 'дать', 'номер', 'звонить', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'г', 'месяц', 'написать', 'проблема', 'работать', 'средство', 'рубль', 'работа', 'документ', 'ситуация', 'звонок', 'прийти', 'решить', 'говорить', 'человек', 'хотеть', 'операция', 'почему', 'должный', 'информация', 'сделать', 'смс', 'срок', 'услуга', 'заявка', 'знать', 'просто', 'обслуживание', 'очередь', 'минута', 'обратиться', 'данный', 'большой', 'первый', 'сегодня', 'девушка', 'ждать', 'сайт', 'руб', 'претензия', 'новый', 'просить', 'интернет', 'процент', 'неделя', 'никакой', 'оператор', 'сообщить', 'ответить', 'являться', 'далее', 'стать', 'нужно', 'час', 'несколько', 'оформить', 'перевод', 'писать', 'комиссия', 'следу

In [66]:
top_words_vec = model_response.wv[top_words]
tsne = TSNE(n_components=2, random_state=21)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [67]:
output_notebook()

p = figure(tools = 'pan, wheel_zoom, reset, save',
           toolbar_location = 'above',
           title = 'TSNE для топ-100 слов')

source = ColumnDataSource(data = dict(x=top_words_tsne[:,0],
                                      y=top_words_tsne[:,1],
                                      names=top_words))

p.scatter(x='x', y='y', size=6, source=source)

labels = LabelSet(x='x', y='y', text = 'names', y_offset = 6,
                  text_font_size = '8pt', text_color = 'blue',
                  source = source, text_align = 'center')
p.add_layout(labels)

show(p)

Loading BokehJS ...


Примеры осмысленных кластеров

![Временные_кластеры](./plots/Data_TSNE-plot.png)

![Денежный_кластер](./plots/sum_TSNE-plot.png)

- задать координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.

In [ ]:
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)
    
    
    
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)